In [3]:
!pip install gradio transformers torch sentence-transformers PyPDF2
import gradio as gr
import PyPDF2
import os
from sentence_transformers import SentenceTransformer, util
import torch
import re

# Initialize the sentence transformer model for embeddings (simulating GPT-3.5-like behavior)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Store document embeddings and content
documents = []
doc_embeddings = []

# Function to extract text from uploaded files (supports PDF and TXT)
def extract_text_from_file(file):
    if file.name.endswith('.pdf'):
        pdf_reader = PyPDF2.PdfReader(file.name)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text
    elif file.name.endswith('.txt'):
        with open(file.name, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

# Function to preprocess and split text into chunks
def preprocess_text(text):
    # Basic cleaning: remove extra whitespace and split into sentences
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = text.split('. ')
    return [s.strip() + '.' for s in sentences if s.strip()]

# Function to index a document
def index_document(file):
    global documents, doc_embeddings
    text = extract_text_from_file(file)
    if not text:
        return "Error: Could not extract text from the file."

    # Split into chunks (sentences)
    chunks = preprocess_text(text)
    documents.extend(chunks)

    # Generate embeddings for each chunk
    embeddings = model.encode(chunks, convert_to_tensor=True)
    doc_embeddings.append(embeddings)

    return f"Indexed document: {os.path.basename(file.name)} ({len(chunks)} chunks)"

# Function to search documents
def search_documents(query):
    if not documents:
        return "No documents indexed yet. Please upload a document first."

    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity between query and document chunks
    all_embeddings = torch.cat(doc_embeddings, dim=0)
    scores = util.cos_sim(query_embedding, all_embeddings)[0]

    # Get top 3 most relevant chunks
    top_k = min(3, len(documents))
    top_results = torch.topk(scores, k=top_k)
    result_indices = top_results.indices.tolist()
    result_scores = top_results.values.tolist()

    # Format the results
    results = []
    for idx, score in zip(result_indices, result_scores):
        results.append(f"Score: {score:.4f} - {documents[idx]}")

    return "\n\n".join(results)

# Clear indexed documents (optional reset functionality)
def clear_index():
    global documents, doc_embeddings
    documents = []
    doc_embeddings = []
    return "Index cleared."

# Gradio Interface
with gr.Blocks(title="AI-Powered Document Search") as demo:
    gr.Markdown("# AI-Powered Document Search and Retrieval System")
    gr.Markdown("Upload documents (PDF or TXT) and search through them using natural language queries.")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Document (PDF or TXT)")
            upload_btn = gr.Button("Index Document")
            clear_btn = gr.Button("Clear Index")
            upload_output = gr.Textbox(label="Upload Status")

        with gr.Column():
            query_input = gr.Textbox(label="Enter your search query")
            search_btn = gr.Button("Search")
            search_output = gr.Textbox(label="Search Results", lines=10)

    # Connect functions to buttons
    upload_btn.click(fn=index_document, inputs=file_input, outputs=upload_output)
    clear_btn.click(fn=clear_index, inputs=None, outputs=upload_output)
    search_btn.click(fn=search_documents, inputs=query_input, outputs=search_output)

# Launch the interface
demo.launch()
import os
from sentence_transformers import SentenceTransformer, util
import torch
import re

# Initialize the sentence transformer model for embeddings (simulating GPT-3.5-like behavior)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Store document embeddings and content
documents = []
doc_embeddings = []

# Function to extract text from uploaded files (supports PDF and TXT)
def extract_text_from_file(file):
    if file.name.endswith('.pdf'):
        pdf_reader = PyPDF2.PdfReader(file.name)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text
    elif file.name.endswith('.txt'):
        with open(file.name, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

# Function to preprocess and split text into chunks
def preprocess_text(text):
    # Basic cleaning: remove extra whitespace and split into sentences
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = text.split('. ')
    return [s.strip() + '.' for s in sentences if s.strip()]

# Function to index a document
def index_document(file):
    global documents, doc_embeddings
    text = extract_text_from_file(file)
    if not text:
        return "Error: Could not extract text from the file."

    # Split into chunks (sentences)
    chunks = preprocess_text(text)
    documents.extend(chunks)

    # Generate embeddings for each chunk
    embeddings = model.encode(chunks, convert_to_tensor=True)
    doc_embeddings.append(embeddings)

    return f"Indexed document: {os.path.basename(file.name)} ({len(chunks)} chunks)"

# Function to search documents
def search_documents(query):
    if not documents:
        return "No documents indexed yet. Please upload a document first."

    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity between query and document chunks
    all_embeddings = torch.cat(doc_embeddings, dim=0)
    scores = util.cos_sim(query_embedding, all_embeddings)[0]

    # Get top 3 most relevant chunks
    top_k = min(3, len(documents))
    top_results = torch.topk(scores, k=top_k)
    result_indices = top_results.indices.tolist()
    result_scores = top_results.values.tolist()

    # Format the results
    results = []
    for idx, score in zip(result_indices, result_scores):
        results.append(f"Score: {score:.4f} - {documents[idx]}")

    return "\n\n".join(results)

# Clear indexed documents (optional reset functionality)
def clear_index():
    global documents, doc_embeddings
    documents = []
    doc_embeddings = []
    return "Index cleared."

# Gradio Interface
with gr.Blocks(title="AI-Powered Document Search") as demo:
    gr.Markdown("# AI-Powered Document Search and Retrieval System")
    gr.Markdown("Upload documents (PDF or TXT) and search through them using natural language queries.")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Document (PDF or TXT)")
            upload_btn = gr.Button("Index Document")
            clear_btn = gr.Button("Clear Index")
            upload_output = gr.Textbox(label="Upload Status")

        with gr.Column():
            query_input = gr.Textbox(label="Enter your search query")
            search_btn = gr.Button("Search")
            search_output = gr.Textbox(label="Search Results", lines=10)

    # Connect functions to buttons
    upload_btn.click(fn=index_document, inputs=file_input, outputs=upload_output)
    clear_btn.click(fn=clear_index, inputs=None, outputs=upload_output)
    search_btn.click(fn=search_documents, inputs=query_input, outputs=search_output)

# Launch the interface
demo.launch()
import os
from sentence_transformers import SentenceTransformer, util
import torch
import re

# Initialize the sentence transformer model for embeddings (simulating GPT-3.5-like behavior)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Store document embeddings and content
documents = []
doc_embeddings = []

# Function to extract text from uploaded files (supports PDF and TXT)
def extract_text_from_file(file):
    if file.name.endswith('.pdf'):
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        return text
    elif file.name.endswith('.txt'):
        with open(file.name, 'r', encoding='utf-8') as f:
            return f.read()
    return ""

# Function to preprocess and split text into chunks
def preprocess_text(text):
    # Basic cleaning: remove extra whitespace and split into sentences
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = text.split('. ')
    return [s.strip() + '.' for s in sentences if s.strip()]

# Function to index a document
def index_document(file):
    global documents, doc_embeddings
    text = extract_text_from_file(file)
    if not text:
        return "Error: Could not extract text from the file."

    # Split into chunks (sentences)
    chunks = preprocess_text(text)
    documents.extend(chunks)

    # Generate embeddings for each chunk
    embeddings = model.encode(chunks, convert_to_tensor=True)
    doc_embeddings.append(embeddings)

    return f"Indexed document: {os.path.basename(file.name)} ({len(chunks)} chunks)"

# Function to search documents
def search_documents(query):
    if not documents:
        return "No documents indexed yet. Please upload a document first."

    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity between query and document chunks
    all_embeddings = torch.cat(doc_embeddings, dim=0)
    scores = util.cos_sim(query_embedding, all_embeddings)[0]

    # Get top 3 most relevant chunks
    top_k = min(3, len(documents))
    top_results = torch.topk(scores, k=top_k)
    result_indices = top_results.indices.tolist()
    result_scores = top_results.values.tolist()

    # Format the results
    results = []
    for idx, score in zip(result_indices, result_scores):
        results.append(f"Score: {score:.4f} - {documents[idx]}")

    return "\n\n".join(results)

# Clear indexed documents (optional reset functionality)
def clear_index():
    global documents, doc_embeddings
    documents = []
    doc_embeddings = []
    return "Index cleared."

# Gradio Interface
with gr.Blocks(title="AI-Powered Document Search") as demo:
    gr.Markdown("# AI-Powered Document Search and Retrieval System")
    gr.Markdown("Upload documents (PDF or TXT) and search through them using natural language queries.")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Document (PDF or TXT)")
            upload_btn = gr.Button("Index Document")
            clear_btn = gr.Button("Clear Index")
            upload_output = gr.Textbox(label="Upload Status")

        with gr.Column():
            query_input = gr.Textbox(label="Enter your search query")
            search_btn = gr.Button("Search")
            search_output = gr.Textbox(label="Search Results", lines=10)

    # Connect functions to buttons
    upload_btn.click(fn=index_document, inputs=file_input, outputs=upload_output)
    clear_btn.click(fn=clear_index, inputs=None, outputs=upload_output)
    search_btn.click(fn=search_documents, inputs=query_input, outputs=search_output)

# Launch the interface
demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6afa67ebbfada92776.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c8d884f4b3e0dbf4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6ecdb56805e74770b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
